## 주식 가격 수집 및 저장 (대성에너지)
+ 저장된 데이터 마지막 일자의 다음날부터 현재일 기준 직전일까지 데이터를 일괄로 가져와서 저장
+ 작성: 임경호

#### 0. 데이터를 가져올 날짜 설정

In [1]:
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect
import myutil

stock_code = '117580'   # 대성에너지 stock code

conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

query = f'SELECT tr_date FROM stock_price where stock_code = {stock_code} ORDER BY tr_date DESC LIMIT 1'
cur.execute(query)

row = cur.fetchone()
last_date = row[0]     # 저장된 데이터의 마지막 일자

conn.close()

start_date = myutil.get_next_day(last_date)
end_date = myutil.get_previous_day()

In [2]:
stock_code, last_date, start_date, end_date

('117580', '20230712', '20230713', '20230726')

#### 1. 주가 데이터 가져오기

In [3]:
from pykrx import stock

# 특정기간 특정 종목의 OHLCV
ohlcv = stock.get_market_ohlcv(start_date, end_date, stock_code)

In [4]:
ohlcv.shape

(10, 7)

In [5]:
ohlcv

,시가,고가,저가,종가,거래량,거래대금,등락률
날짜,,,,,,,
2023-07-13,7830,7940,7830,7840,46989,369399500,0.13
2023-07-14,7870,7870,7740,7790,42639,331877980,-0.64
2023-07-17,7750,7900,7730,7800,50242,391124830,0.13
2023-07-18,7770,7850,7710,7740,42671,331198060,-0.77
2023-07-19,7810,7810,7650,7700,52199,403211990,-0.52
2023-07-20,7600,7720,7600,7680,34671,264982300,-0.26
2023-07-21,7740,7830,7670,7670,63134,486972870,-0.13
2023-07-24,7670,7700,7370,7370,100278,749267830,-3.91
2023-07-25,7380,7410,7110,7120,121452,874114900,-3.39


#### 2. 데이터 변환 및 전처리

In [6]:
df_ohlcv = ohlcv.reset_index()
df_ohlcv

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률
0,2023-07-13,7830,7940,7830,7840,46989,369399500,0.13
1,2023-07-14,7870,7870,7740,7790,42639,331877980,-0.64
2,2023-07-17,7750,7900,7730,7800,50242,391124830,0.13
3,2023-07-18,7770,7850,7710,7740,42671,331198060,-0.77
4,2023-07-19,7810,7810,7650,7700,52199,403211990,-0.52
5,2023-07-20,7600,7720,7600,7680,34671,264982300,-0.26
6,2023-07-21,7740,7830,7670,7670,63134,486972870,-0.13
7,2023-07-24,7670,7700,7370,7370,100278,749267830,-3.91
8,2023-07-25,7380,7410,7110,7120,121452,874114900,-3.39
9,2023-07-26,7010,7120,6520,6780,224164,1513059790,-4.78


In [7]:
# 날짜 형식 변환 => 문자열 yyyymmdd
df_ohlcv['날짜'] = df_ohlcv['날짜'].astype('str')
df_ohlcv['날짜'] = df_ohlcv['날짜'].str.replace('-', '')

In [8]:
df_ohlcv

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률
0,20230713,7830,7940,7830,7840,46989,369399500,0.13
1,20230714,7870,7870,7740,7790,42639,331877980,-0.64
2,20230717,7750,7900,7730,7800,50242,391124830,0.13
3,20230718,7770,7850,7710,7740,42671,331198060,-0.77
4,20230719,7810,7810,7650,7700,52199,403211990,-0.52
5,20230720,7600,7720,7600,7680,34671,264982300,-0.26
6,20230721,7740,7830,7670,7670,63134,486972870,-0.13
7,20230724,7670,7700,7370,7370,100278,749267830,-3.91
8,20230725,7380,7410,7110,7120,121452,874114900,-3.39
9,20230726,7010,7120,6520,6780,224164,1513059790,-4.78


#### 3. DB 테이블에 저장

In [9]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_ohlcv.itertuples():
    sql = "insert into stock_price (stock_code, tr_date, open, high, low, close, volume, tr_amount, fluctuation_rate) \
                            values (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (stock_code, row[1], row[2], row[3], row[4], row[5], row[6], row[7], round(row[8],2)))
conn.commit()
conn.close()